In [49]:
import pandas as pd
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import numpy as np

In [50]:
def corriger_encodage(df):
    # Fonction pour corriger l'encodage d'une chaîne de caractères
    def corriger_chaine(chaine):
        if isinstance(chaine, str):
            try:
                return chaine.encode('latin1').decode('utf-8')
            except UnicodeEncodeError:
                return chaine
        return chaine

    # Corriger les valeurs dans le DataFrame
    for col in df.columns:
        df[col] = df[col].apply(corriger_chaine)

    # Corriger les noms de colonnes
    df.columns = [corriger_chaine(col) for col in df.columns]

    return df

In [51]:
df = corriger_encodage(pd.read_csv('./data/traffic_to_ml.csv', encoding='unicode_escape').drop(columns=["Unnamed: 0"]))

In [52]:
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

In [53]:
df.columns

Index(['Year', 'Area', 'Date', 'Road', 'Municipality Name', 'County Name',
       'Province Name', 'Fatalties', 'Serious Injuries', 'Light Injuries',
       'Total Victims', 'Units Involved', 'Pedestrains Involved',
       'Bicycles Involved', 'Mopeds Involved', 'Motorcycles',
       'Light Vehicles Involved', 'Heavy Vehicles Involved',
       'Other Units Involved', 'Unspecified Units Involved',
       'Road Speed Limit', 'Accident with Hit and Run', 'Fog Presence',
       'Surrounding Environment', 'Special Lane Presence',
       'Special Traffic Measures', 'Weather Conditions',
       'Special Road Functions', 'Severity of Accident', 'Influence of Fog',
       'Influence of Environment', 'Influence of Traffic',
       'Influence of Weather', 'Influence of Wind Intensity',
       'Influence of Lighting', 'Influence of Special Measures',
       'Influence of Road Objects', 'Influence of Road Surface',
       'Influence of Visibility', 'Intersection Characteristics',
       'Lighting C

# FATALTIES

In [54]:
X_fatalties = df[numerical_cols].drop(['Fatalties', 'Total Victims'], axis=1)  # Drop non-feature columns
y_fatalties = df[numerical_cols]['Fatalties']  #

In [55]:
X_train_fatalties, X_test_fatalties, y_train_fatalties, y_test_fatalties = train_test_split(X_fatalties, y_fatalties, test_size=0.2, random_state=42)  # Notice shuffle=False for time series

In [56]:
scaler = StandardScaler()
X_train_scaled_fatalties = scaler.fit_transform(X_train_fatalties)
X_test_scaled_fatalties = scaler.transform(X_test_fatalties)

In [57]:
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models_fatalties, predictions_fatalties = reg.fit(X_train_scaled_fatalties, X_test_scaled_fatalties, y_train_fatalties, y_test_fatalties)

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:01<?, ?it/s]


KeyboardInterrupt: 

# Serious Injuries

In [ ]:
X_serious = df[numerical_cols].drop(['Serious Injuries', 'Total Victims'], axis=1)  # Drop non-feature columns
y_serious = df[numerical_cols]['Serious Injuries']  #

In [ ]:
X_train_serious, X_test_serious, y_train_serious, y_test_serious = train_test_split(X_serious, y_serious, test_size=0.2, random_state=42)  # Notice shuffle=False for time series

In [ ]:
scaler = StandardScaler()
X_train_scaled_serious = scaler.fit_transform(X_train_serious)
X_test_scaled_serious = scaler.transform(X_test_serious)

In [ ]:
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models_serious, predictions_serious = reg.fit(X_train_scaled_serious, X_test_scaled_serious, y_train_serious, y_test_serious)

100%|██████████| 42/42 [12:45<00:00,  7.07s/it] 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003164 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 578
[LightGBM] [Info] Number of data points in the train set: 16881, number of used features: 48
[LightGBM] [Info] Start training from score 0.999171


100%|██████████| 42/42 [12:45<00:00, 18.22s/it]


# Light Injuries

In [ ]:
X_light = df[numerical_cols].drop(['Light Injuries', 'Total Victims'], axis=1)  # Drop non-feature columns
y_light  = df[numerical_cols]['Light Injuries']  #

In [ ]:
X_train_light, X_test_light, y_train_light, y_test_light = train_test_split(X_light, y_light, test_size=0.2, random_state=42)  # Notice shuffle=False for time series

In [ ]:
scaler = StandardScaler()
X_train_scaled_light = scaler.fit_transform(X_train_light)
X_test_scaled_light = scaler.transform(X_test_light)

In [ ]:
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models_light, predictions_light = reg.fit(X_train_scaled_light, X_test_scaled_light, y_train_light, y_test_light)

 95%|█████████▌| 40/42 [03:57<00:08,  4.26s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001882 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 573
[LightGBM] [Info] Number of data points in the train set: 16881, number of used features: 48
[LightGBM] [Info] Start training from score 0.398910


100%|██████████| 42/42 [03:57<00:00,  5.66s/it]


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Year

In [ ]:
yearly_accidents = df.groupby('Year').size()

# Split the data (assuming you have multiple years, and you leave one year for testing)
train_data = yearly_accidents[:-1]
test_data = yearly_accidents[-1:]

In [ ]:
# You may need to find the best parameters (p, d, q) (P, D, Q, s) using grid search or AIC
sarima_model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))  # these are example parameters
sarima_result = sarima_model.fit()

# Forecast
sarima_forecast = sarima_result.get_forecast(steps=1)  # forecast next year
sarima_forecast.summary_frame()

C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


y,mean,mean_se,mean_ci_lower,mean_ci_upper
11,1208.00,1000.00,-751.96,3167.96


In [ ]:
# This is Holt-Winters method with additive trend and seasonality
hw_model = ExponentialSmoothing(train_data, trend='add', seasonal='add', seasonal_periods=4).fit()  # adjust seasonal_periods based on your data's seasonality

# Forecast
hw_forecast = hw_model.forecast(steps=1)  # forecast next year

C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
# For SARIMA
sarima_pred = sarima_result.predict(start=test_data.index[0], end=test_data.index[0])
print('SARIMA prediction:', sarima_pred)
print('Actual:', test_data)

# For Exponential Smoothing
print('Holt-Winters prediction:', hw_forecast)

SARIMA prediction: 2021   39104.08
dtype: float64
Actual: Year
2021    1446
dtype: int64
Holt-Winters prediction: 11   1338.08
dtype: float64


C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


# Month

In [ ]:
monthly_accidents = df.groupby('Month').size()

# Split the data (assuming you have multiple years, and you leave one year for testing)
train_data = monthly_accidents[:-1]
test_data = monthly_accidents[-1:]

In [ ]:
# You may need to find the best parameters (p, d, q) (P, D, Q, s) using grid search or AIC
sarima_model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))  # these are example parameters
sarima_result = sarima_model.fit()

# Forecast
sarima_forecast = sarima_result.get_forecast(steps=1)  # forecast next year
sarima_forecast.summary_frame()

C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


y,mean,mean_se,mean_ci_lower,mean_ci_upper
11,1736.00,1000.00,-223.96,3695.96


In [ ]:
# This is Holt-Winters method with additive trend and seasonality
hw_model = ExponentialSmoothing(train_data, trend='add', seasonal='add', seasonal_periods=2).fit()  # adjust seasonal_periods based on your data's seasonality

# Forecast
hw_forecast = hw_model.forecast(steps=12)  # forecast next year

C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
# For SARIMA
sarima_pred = sarima_result.predict(start=test_data.index[0], end=test_data.index[0])
print('SARIMA prediction:', sarima_pred)
print('Actual:', test_data)

# For Exponential Smoothing
print('Holt-Winters prediction:', hw_forecast)

SARIMA prediction: 12   2529.00
dtype: float64
Actual: Month
12    1587
dtype: int64
Holt-Winters prediction: 11   1890.07
12   1929.39
13   1945.19
14   1984.51
15   2000.32
16   2039.63
17   2055.44
18   2094.75
19   2110.56
20   2149.87
21   2165.68
22   2204.99
dtype: float64


C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


# Day

In [ ]:
day_accidents = df.groupby('Day').size()

# Split the data (assuming you have multiple years, and you leave one year for testing)
train_data = day_accidents[:-1]
test_data = day_accidents[-1:]

In [ ]:
# You may need to find the best parameters (p, d, q) (P, D, Q, s) using grid search or AIC
sarima_model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))  # these are example parameters
sarima_result = sarima_model.fit()

# Forecast
sarima_forecast = sarima_result.get_forecast(steps=1)  # forecast next year
sarima_forecast.summary_frame()

C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


y,mean,mean_se,mean_ci_lower,mean_ci_upper
30,658.79,27.12,605.63,711.95


In [ ]:
# This is Holt-Winters method with additive trend and seasonality
hw_model = ExponentialSmoothing(train_data, trend='add', seasonal='add', seasonal_periods=7).fit()  # adjust seasonal_periods based on your data's seasonality

# Forecast
hw_forecast = hw_model.forecast(steps=30)  # forecast next year

C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
# For SARIMA
sarima_pred = sarima_result.predict(start=test_data.index[0], end=test_data.index[0])
print('SARIMA prediction:', sarima_pred)
print('Actual:', test_data)

# For Exponential Smoothing
print('Holt-Winters prediction:', hw_forecast)

SARIMA prediction: 31   665.15
dtype: float64
Actual: Day
31    408
dtype: int64
Holt-Winters prediction: 30   692.96
31   677.05
32   699.44
33   693.56
34   706.83
35   682.55
36   685.52
37   693.24
38   677.33
39   699.71
40   693.84
41   707.11
42   682.83
43   685.80
44   693.52
45   677.60
46   699.99
47   694.11
48   707.39
49   683.11
50   686.07
51   693.79
52   677.88
53   700.26
54   694.39
55   707.66
56   683.38
57   686.35
58   694.07
59   678.15
dtype: float64


C:\Users\hugoz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


# RESULT

In [ ]:
display(models_fatalties)
display(models_serious)
display(models_light)

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
HistGradientBoostingRegressor,0.89,0.89,0.13,1.13
XGBRegressor,0.88,0.89,0.13,0.45
LGBMRegressor,0.88,0.88,0.13,0.36
ElasticNetCV,0.88,0.88,0.13,0.44
LarsCV,0.88,0.88,0.13,0.57
LassoLarsCV,0.88,0.88,0.13,0.31
LassoCV,0.88,0.88,0.13,0.43
LassoLarsIC,0.88,0.88,0.13,0.25
OrthogonalMatchingPursuit,0.88,0.88,0.13,0.06


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
LGBMRegressor,0.41,0.42,0.37,0.30
HistGradientBoostingRegressor,0.40,0.41,0.38,0.74
XGBRegressor,0.36,0.37,0.39,0.27
RandomForestRegressor,0.34,0.34,0.40,13.25
ExtraTreesRegressor,0.33,0.34,0.40,9.02
SVR,0.33,0.34,0.40,10.51
LassoLarsIC,0.29,0.30,0.41,0.15
BayesianRidge,0.29,0.30,0.41,0.09
ElasticNetCV,0.29,0.30,0.41,1.07


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
HistGradientBoostingRegressor,0.22,0.22,0.92,0.49
LGBMRegressor,0.21,0.22,0.92,0.52
GradientBoostingRegressor,0.20,0.21,0.93,2.82
XGBRegressor,0.19,0.20,0.93,0.19
RandomForestRegressor,0.19,0.20,0.93,16.40
SVR,0.15,0.16,0.95,19.96
ExtraTreesRegressor,0.15,0.16,0.95,13.65
NuSVR,0.14,0.15,0.96,56.18
LassoLarsIC,0.13,0.14,0.96,0.11


In [85]:
num_features = len(numerical_cols) - 1

In [86]:
model = Sequential()
model.add(Dense(128, input_dim=num_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Output layer for regression

# Output layer - adjust according to your problem
# For regression: No activation function
# For binary classification: 1 neuron, 'sigmoid' activation
# For multi-class classification: n neurons (n = number of classes), 'softmax' activation
# Compile the model
model.compile(optimizer = 'rmsprop',
                loss = 'mse',
                metrics = ['mae'])
# Model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 128)               6528      
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 1)                 33        
                                                                 
Total params: 16897 (66.00 KB)
Trainable params: 16897

In [82]:
data       = df[numerical_cols].sample(frac=1., axis=0)
data_train = data.sample(frac=0.7, axis=0)
data_test  = data.drop(data_train.index)

# ---- Split => x,y (medv is price)
#
x_train = data_train.drop('Fatalties',  axis=1)
y_train = data_train['Fatalties']
x_test  = data_test.drop('Fatalties',   axis=1)
y_test  = data_test['Fatalties']

In [83]:
mean = x_train.mean()
std  = x_train.std()
x_train = (x_train - mean)/std
x_test  = (x_test - mean)/std

x_train, y_train = np.array(x_train), np.array(y_train)
x_test,  y_test  = np.array(x_test), np.array(y_test)

In [87]:
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=60,
    validation_data = (x_test, y_test)
)

Epoch 1/60
462/462 [==============================] - 2s 2ms/step - loss: 0.1527 - mae: 0.2187 - val_loss: 0.1136 - val_mae: 0.2310
Epoch 2/60
462/462 [==============================] - 1s 2ms/step - loss: 0.1507 - mae: 0.2378 - val_loss: 0.1138 - val_mae: 0.2393
Epoch 3/60
462/462 [==============================] - 1s 2ms/step - loss: 0.1507 - mae: 0.2413 - val_loss: 0.1136 - val_mae: 0.2288
Epoch 4/60
462/462 [==============================] - 1s 2ms/step - loss: 0.1507 - mae: 0.2388 - val_loss: 0.1137 - val_mae: 0.2354
Epoch 5/60
462/462 [==============================] - 1s 2ms/step - loss: 0.1507 - mae: 0.2408 - val_loss: 0.1136 - val_mae: 0.2281
Epoch 6/60
462/462 [==============================] - 1s 2ms/step - loss: 0.1507 - mae: 0.2390 - val_loss: 0.1136 - val_mae: 0.2329
Epoch 7/60
462/462 [==============================] - 1s 2ms/step - loss: 0.1507 - mae: 0.2407 - val_loss: 0.1136 - val_mae: 0.2261
Epoch 8/60
462/462 [==============================] - 1s 2ms/step - loss: 0.

In [88]:
score = history.history["mae"]

print('x_test / loss      : {:5.4f}'.format(score[0]))
print('x_test / mae       : {:5.4f}'.format(score[1]))

x_test / loss      : 0.2187
x_test / mae       : 0.2378


In [89]:
print(history.history.keys())

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])


In [90]:
predictions = model.predict(x_test)

7/7 [==============================] - 0s 3ms/step


In [91]:
print("Prediction : {:.2f} K$".format(predictions[0][0]))

Prediction : 0.14 K$


In [ ]:
df[numerical_cols].describe()

,Year,Area,Province Name,Fatalties,Serious Injuries,Light Injuries,Total Victims,Units Involved,Pedestrains Involved,Bicycles Involved,...,Type of Road,Road Ownership,Wind Conditions,Day of the Week Grouping,Hour of Day,Time of Day Grouping,Type of Accident,Day Type,Day,Month
count,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,...,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00,21102.00
mean,2015.15,0.55,2.20,0.14,1.00,0.40,1.53,1.89,0.25,0.09,...,4.36,0.44,1.98,0.64,13.85,0.71,2.16,3.05,15.81,6.60
std,3.37,0.50,1.12,0.40,0.53,0.97,1.19,0.78,0.50,0.33,...,0.89,1.69,0.18,0.48,5.37,0.67,1.18,1.98,8.79,3.35
min,2010.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,...,0.00,-1.00,-1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
25%,2012.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,0.00,...,4.00,-1.00,2.00,0.00,10.16,0.00,2.00,1.00,8.00,4.00
50%,2015.00,1.00,3.00,0.00,1.00,0.00,1.00,2.00,0.00,0.00,...,5.00,-1.00,2.00,1.00,14.15,1.00,2.00,3.00,16.00,7.00
75%,2018.00,1.00,3.00,0.00,1.00,0.00,2.00,2.00,0.00,0.00,...,5.00,2.00,2.00,1.00,18.25,1.00,3.00,5.00,23.00,10.00
max,2021.00,1.00,3.00,13.00,23.00,25.00,49.00,21.00,10.00,8.00,...,5.00,4.00,2.00,1.00,23.59,2.00,5.00,6.00,31.00,12.00


In [58]:
df['Date'] = pd.to_datetime(df['Date'])  # Ensure 'Date' is a datetime object
df.set_index('Date', inplace=True)  # Set 'Date' as the index

monthly_accidents = df.resample('M').size()

In [61]:
df.head()

,Year,Area,Road,Municipality Name,County Name,Province Name,Fatalties,Serious Injuries,Light Injuries,Total Victims,...,Type of Road,Road Ownership,Wind Conditions,Day of the Week Grouping,Hour of Day,Time of Day Grouping,Type of Accident,Day Type,Day,Month
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-25,2010,1,SE,CANOVES I SAMALUS,Valles Oriental,3,0,1,0,1,...,5,-1,2,1,23.33,2,2,0,25,1
2010-10-31,2010,0,N-240,LLEIDA,Segria,2,0,1,3,4,...,4,1,2,0,1.00,2,1,6,31,10
2010-05-17,2010,0,N-II,FORNELLS DE LA SELVA,Girones,1,1,0,2,3,...,4,1,2,1,15.27,1,2,0,17,5
2010-08-21,2010,1,SE,BARCELONA,Barcelones,3,0,2,7,9,...,5,-1,2,0,22.30,2,2,5,21,8
2010-05-07,2010,1,SE,BADALONA,Barcelones,3,0,1,0,1,...,5,-1,2,0,17.45,1,0,4,7,5


In [65]:
import pmdarima as pm

In [75]:
train = monthly_accidents[:'2018-01-01']  # Replace with your split date
test = monthly_accidents['2018-01-01':]  # Replace with your split date

# Build and fit the ARIMA model
model = pm.auto_arima(monthly_accidents, seasonal=True, m=1,
                      start_p=0, start_q=0, 
                      max_p=10, max_q=10, 
                      start_P=0, start_Q=0, 
                      max_P=10, max_Q=10, 
                      d=None, D=None, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1320.489, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1308.285, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=1300.348, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=1318.505, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=1289.273, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.20 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=1285.273, Time=0.11 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.27 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.31 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=1286.173, Time=0.07 sec

Best model:  ARIMA(0,1,3)(0,0,0)[0] intercept
Total fit time: 1.568 seconds


In [76]:
print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  144
Model:               SARIMAX(0, 1, 3)   Log Likelihood                -637.637
Date:                Wed, 24 Jan 2024   AIC                           1285.273
Time:                        23:16:12   BIC                           1300.088
Sample:                    01-31-2010   HQIC                          1291.293
                         - 12-31-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.3423      0.135     -2.537      0.011      -0.607      -0.078
ma.L1         -0.5489      0.064     -8.571      0.000      -0.674      -0.423
ma.L2         -0.1981      0.093     -2.133      0.0

In [77]:
model.predict()

2022-01-31   111.83
2022-02-28   117.15
2022-03-31   117.42
2022-04-30   117.08
2022-05-31   116.74
2022-06-30   116.39
2022-07-31   116.05
2022-08-31   115.71
2022-09-30   115.37
2022-10-31   115.02
Freq: M, dtype: float64

146.54166666666666